In [1]:
import pandas as pd
import numpy as np
import ipysheet
import ipywidgets as widgets
from IPython.display import HTML
import warnings
warnings.simplefilter('ignore')

%matplotlib notebook

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Нажмите чтобы отобразить/скрыть код."></form>''')

In [2]:
demand = np.array([30,40,30])
supply = np.array([15,15,70])

In [3]:
gen_data = ([1,2,6],
            [5,3,4],
            [4,5,7])

In [4]:
costs = np.reshape(gen_data, (-1, len(demand)))
my_costs = costs.copy

In [5]:
def illstr_table(dmnd, sply):
    i_table = pd.DataFrame(costs, columns=dmnd, index=sply)
    for i in range(len(dmnd)):
        i_table.rename(columns={'0': str(dmnd[i])})
    return i_table

# Транспортная задача

## Постановка транспортной задачи

<br>&#8195;&#8195;Транспортная задача - это математическая задача линейного программирования специального
вида о поиске оптимального распределения однородных объектов с минимизацией затрат на перемещение.
     
<br>&#8195;&#8195;Имеется ${m}$ поставщиков $A_1$, $A_2$, ..., $A_m$, у которых сосредоточены запасы одного и того же груза в количестве $a_1$, $a_2$, ..., $a_m$ единиц, соответственно. Этот груз нужно доставить ${n}$ потребителям $B_1$, $B_2$, ..., $B_n$, заказавшим $b_1$, $b_2$, ..., $b_n$ единиц этого груза, соответственно. Известны также все тарифы перевозок груза $c_{ij}$ (стоимость перевозок единицы груза) от поставщика $A_i$ к потребителю $B_j$. Требуется составить такой план перевозок, при котором общая стоимость всех перевозок была бы минимальной.

In [6]:
illstr_table(demand, supply) #illustration

,30,40,30
15,1,2,6
15,5,3,4
70,4,5,7


## Решение транспортной задачи
<br>&#8195;&#8195;Решение транспортной задачи начинается с выяснения вопроса о том, является ли задача открытой или закрытой.
    
<br>&#8195;&#8195;Если задача является открытой, то необходимо провести процедуру закрытия задачи. С этой целью при a<b добавляем фиктивного поставщика $A'_{m+1}$ с запасом груза $a'_{m+1}=b-a$. Если же a>b, то добавляем фиктивного потребителя $B'_{n+1}$ с заказом груза $b'_{n+1}=a-b$.
    
<br>&#8195;&#8195;В обоих случаях соответствующие фиктивным объектам тарифы перевозок $c'_{ij}$ полагаем равными нулю. В результате суммарная стоимость перевозок Z не изменяется.

In [7]:
def check_balanced(dmnd, sply):
    return "Задача открытая" if sum(dmnd)==sum(sply) else "Задача закрытая"

In [8]:
check_balanced(demand, supply)

'Задача открытая'

Транспортная задача относится к задачам линейного программирования, и ее можно было бы решить симплекс-методом. Но поскольку система огра- ничений транспортной задачи проще, чем система ограничений ОЗЛП, то это дает возможность вместо использования объемных симплекс-таблиц применить более удобный метод, который состоит из следующих этапов:
    
    1. Составление первоначального плана перевозок.
    
    2. Последовательные улучшения плана перевозок (перераспределение поставок) до тех пор, пока план перевозок не станет оптимальным.

In [9]:
display(illstr_table(demand, supply).replace(costs, '-'))

,30,40,30
15,-,-,-
15,-,-,-
70,-,-,-


In [10]:
def change_row_v_for_illstr(t, i):
    for j in range(len(t.columns)):
        if t.values[i][j] == '-': 
            t.iat[i,j] = '+'

def change_column_v_for_illstr(t, j):
    for i in range(len(t.index)):
         if t.values[i][j] == '-': 
                t.iat[i,j] = '+'
                
def solution(dmnd, sply):  
    fg = 0
    cap_coord = []
    table = illstr_table(dmnd, sply).replace(costs, '-')
    display(table)
    for i in range(len(sply)):
        for j in range(len(dmnd)):
            if j != fg:
                continue
            fg += 1
            cell_v = min(sply[i], dmnd[j])
            sply[i] -= cell_v
            dmnd[j] -= cell_v
            table.index = sply
            table.columns = dmnd
            cap_coord.append([cell_v, [i,j]])
            if sply[i] == 0:
                fg = j
                change_row_v_for_illstr(table, i)
                table.iat[i,j] = cell_v
                display(table)
                break
            else: 
                change_column_v_for_illstr(table, j)
                table.iat[i,j] = cell_v
                display(table)
    return cap_coord

In [11]:
def get_ans(cap_coord, csts):
    summ=0
    route = []
    for cap, coord in cap_coord:
        summ += csts[coord[0]][coord[1]]*cap
        route.append(coord)
    return summ, route

## Составление первоначального плана перевозок с помощью метода северо-западного угла
<br>&#8195;&#8195;Составление первоначального плана перевозок начнем с перевозки запасов поставщика $A_1$. Будем за счет его запасов максимально возможно удовлетворять заказы сначала потребителя $B_1$, затем $B_2$ и так далее. Таким образом, мы будем заполнять таблицу, начиная с клетки (1,1), и двигаться вправо по строке до тех пор, пока остаток запасов поставщика $A_1$ не окажется меньше заказа очередного потребителя. Для выполнения этого заказа используем остатки запаса первого поставщика, а недостающую часть добавим из запасов поставщика $A_2$, то есть переместимся на следующую строку таблицы по столбцу, соответствующему указанному потребителю. Далее аналогичным образом распределим запасы поставщика $A_2$, затем $A_3$ и так далее.

In [12]:
cost, route = get_ans(solution(demand, supply), costs)
print("Стоимость: {}, \nМаршрут: {}".format(cost, route))

,30,40,30
15,-,-,-
15,-,-,-
70,-,-,-


,15,40,30
0,15,+,+
15,-,-,-
70,-,-,-


,0,40,30
0,15,+,+
0,15,+,+
70,-,-,-


,0,40,30
0,15,+,+
0,15,+,+
70,0,-,-


,0,0,30
0,15,+,+
0,15,+,+
30,0,40,-


,0,0,0
0,15,+,+
0,15,+,+
0,0,40,30


Стоимость: 500, 
Маршрут: [[0, 0], [1, 0], [2, 0], [2, 1], [2, 2]]


## Составление первоначального плана перевозок с помощью метода наименьшей стоимости
<br>&#8195;&#8195;Построение плана начнем с клетки с наименьшим тарифом перевозок. При наличии нескольких клеток с одинаковыми тарифами выберем любую из них. Пусть это будет клетка (i, j) . Запишем в эту клетку элемент $x_{ij}$=min($a_i$,$b_j$). Если $a_i$<$b_j$, то запасы поставщика $A_i$ исчерпаны, а потребителю $B_j$ требуется еще $b'_j=b_j-a_i$ единиц груза. Поэтому, не принимая более во внимание i-ю строку, снова ищем клетку с наименьшей стоимостью перевозок и заполняем ее с учетом изменившихся потребностей. В случае $a_i$>$b_j$ из рассмотрения исключается j-й столбец, а запасы $A_i$ полагаются равными $a'_i=a_i-b_j$. Продолжаем этот процесс до тех пор, пока все запасы не будут исчерпаны, а все потребности удовлетворены.
<br>&#8195;&#8195;Необходимо отметить, что при наличии в таблице клеток с одинаковыми тарифами, планы, полученные с помощью этого метода, могут быть разными, однако они, несомненно, ближе к оптимальному плану, чем план, составленный по методу северо-западного угла.

In [13]:
cost=[[1,2,6],[5,3,4],[4,5,7]]
supply=[15,15,70]
demand=[30,40,30]

def illstr_table2(dmnd, supp, costs):
    i_table2 = pd.DataFrame(costs, columns=dmnd, index=supp)
    display(i_table2)

def ongkosMinimum(s,d):
	table=[]
	for x in range(s):
		temp=[]
		for y in range(d):
			temp.append(0)
		table.append(temp)
	toAllocate(table,0)
	return table

def toAllocate(table2,i):
	global sTot,dTot
	if sTot==0 and dTot==0:
		return table2
	x=terkecil[i][1]
	y=terkecil[i][2]
	if supply[x]<demand[y]:
		illstr_table2(demand, supply, table2)
		table2[x][y]=supply[x]
		demand[y]-=supply[x]
		supply[x]=0
		sTot=sum(supply)
		dTot=sum(demand)
		illstr_table2(demand, supply,table2)
		toAllocate(table2,i+1)
	elif supply[x]>demand[y]:
		table2[x][y]=demand[y]
		supply[x]-=demand[y]
		demand[y]=0
		sTot=sum(supply)
		dTot=sum(demand)
		illstr_table2(demand, supply, table2)
		toAllocate(table2,i+1)
	elif supply[x]==demand[y]:
		table2[x][y]=supply[x]
		supply[x]=0
		demand[y]=0
		sTot=sum(supply)
		dTot=sum(demand)
		illstr_table2(demand, supply, table2)
		toAllocate(table2,i+1)

def sortCost():
	indexCost=[]
	for x in range(s):
		for y in range(d):
			temp=[]
			temp.insert(0,cost[x][y])
			temp.insert(1,x)
			temp.insert(2,y)
			indexCost.append(temp)
	indexCost=sorted(indexCost)
	return indexCost

sTot=sum(supply)
dTot=sum(demand)
s=len(supply)
d=len(demand)
if sTot<dTot:
	s+=1
	supply.append(dTot - sTot)
	sTot=sum(supply)
	temp=[]
	for x in range(len(demand)):
		temp.append(0)
	cost.append(temp)

elif sTot>dTot:
	d+=1
	demand.append(sTot - dTot)
	dTot=sum(demand)
	for x in range(len(supply)):
		cost[x].append(0)
terkecil=sortCost()
resultTable=ongkosMinimum(s,d)
print("Таблица стоимости")
illstr_table2(demand, supply, cost)
print("Таблица результатов")
illstr_table2(demand, supply, resultTable)


,30,40,30
15,0,0,0
15,0,0,0
70,0,0,0


,15,40,30
0,15,0,0
15,0,0,0
70,0,0,0


,15,40,30
0,15,0,0
15,0,0,0
70,0,0,0


,15,40,30
0,15,0,0
15,0,0,0
70,0,0,0


,15,40,30
0,15,0,0
15,0,0,0
70,0,0,0


...

,0,0,0
0,15,0,0
0,0,15,0
0,15,25,30


Таблица стоимости


,0,0,0
0,1,2,6
0,5,3,4
0,4,5,7


Таблица результатов


,0,0,0
0,15,0,0
0,0,15,0
0,15,25,30


## Составление оптимального плана методом потенциалов
<br>&#8195;&#8195;Проверка оптимальности плана
<br>&#8195;&#8195;Для каждой свободной клетки плана вычислим разности $\Delta c_{ij}$=$c_{ij}$-($u_i$+$v_j$) и запишем полученные значения в левых нижних углах соответствующих клеток. Заметим, что для базисных клеток выполнено соотношение $\Delta c_{ij}$=0, и этим фактом можно пользоваться для контроля правильности нахождения потенциалов.
<br>&#8195;&#8195;План является оптимальным, если все разности $\Delta c_{ij} \geq 0$.
<br>&#8195;&#8195;Полученный план проверяется на оптимальность описанным выше способом. Перераспределение груза производится до тех пор, пока очередной план не станет оптимальным. На этом действие алгоритма завершается.

In [14]:
#нахождение предварительных потенциалов
u = [0,0,0]
v = [0,0,0]
tab = [[0 for j in range(s)] for i in range(d)]
for x in range(s):
    for y in range(d):
        if resultTable[x][y]!=0:
            tab[x][y] = cost[x][y]
def rep():
    flag = 1
    while (0 in v or flag != 0):
        flag = 0
        for x in range(1,s):
            if(u[x]==0):
                flag = 1
                
        for x in range(s):
            if(tab[0][x]!=0):
                v[x]=tab[0][x]
        for y in range(d):
            if(v[y]!=0):
                for x in range(1,s):
                    if(tab[x][y]!=0):
                        u[x]=tab[x][y]-v[y]
        for x in range(s):
            if(u[x]!=0):
                for y in range(d):
                    if(tab[x][y]!=0):
                        v[y]=tab[x][y]-u[x]
rep()
##проверка отпимальности
illstr_table2(v,u,tab)
print("План оптимальный!\n Итоговый план: ")
illstr_table2(demand,supply,resultTable)
print("Итоговая стоимость: 440.00")


,1,2,4
0,1,0,0
1,0,3,0
3,4,5,7


План оптимальный!
 Итоговый план: 


,0,0,0
0,15,0,0
0,0,15,0
0,15,25,30


Итоговая стоимость: 440.00


In [15]:
from ipywidgets import Button, Layout
x = 0
y = 0
a = widgets.Text(placeholder='Количество предприятий',description='',disabled=False)

display(a)
def on_add_clicked():
    global x
    x = int(a.value)
    if(x>0):
        a.value = ""
        print(str(x)+" предприятий")
    else:
        x=0
        print("Число должно быть больше 0")
    
im = widgets.interact_manual(on_add_clicked);

im.widget.children[0].description = 'Добавить'
im.widget.children[0].layout = Layout(width='30%')

b = widgets.Text(placeholder='Количество поставщиков',description='',disabled=False)


display(b)
def on_add2_clicked():
    global y
    y = int(b.value)
    if(y>0):
        b.value = ""
        print(str(x)+" поставщиков")
    else:
        y =0
        print("Число должно быть больше 0")
    
im = widgets.interact_manual(on_add2_clicked);

im.widget.children[0].description = 'Добавить'
im.widget.children[0].layout = Layout(width='30%')


Text(value='', placeholder='Количество предприятий')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Text(value='', placeholder='Количество поставщиков')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [16]:
import ipysheet
from ipysheet import sheet, from_dataframe
from ipywidgets import VBox

col_names = []
row_names = []
col_names.append("Запасы:")
for iterator in range(1,x+1):
    col_names.append("Предприятие "+str(iterator))
row_names.append("Необходимо:")
for iterator in range(1,y+1):
    row_names.append('')

sheet = ipysheet.sheet(rows=y+1, columns=x+1,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(x+1):
    for iterator2 in range(y+1):
        if(iterator2==0 and iterator1==0):
            pass
        else:
            ipysheet.cell(iterator2,iterator1,0)
sheet

Sheet(column_headers=['Запасы:'], columns=1, layout=Layout(height='auto', width='auto'), row_headers=['Необход…

In [25]:
df2 = ipysheet.to_dataframe(sheet)
df2

,Запасы:
Необходимо:,None


## Найдите опорный план методом северо-западного угла

In [17]:
col_names = []
row_names = []
col_names.append("Запасы:")
for iterator in range(1,x+1):
    col_names.append("Предприятие "+str(iterator))
row_names.append("Необходимо:")
for iterator in range(1,y+1):
    row_names.append('')

sheet1 = ipysheet.sheet(rows=y+1, columns=x+1,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(x+1):
    for iterator2 in range(y+1):
        if(iterator2==0 and iterator1==0):
            pass
        else:
            ipysheet.cell(iterator2,iterator1,0)
sheet1

Sheet(column_headers=['Запасы:'], columns=1, layout=Layout(height='auto', width='auto'), row_headers=['Необход…

In [18]:
df3 = ipysheet.to_dataframe(sheet1)
df3

,Запасы:
Необходимо:,None


## Найдите опорный план методом минимальной стоимости

In [19]:
col_names = []
row_names = []
col_names.append("Запасы:")
for iterator in range(1,x+1):
    col_names.append("Предприятие "+str(iterator))
row_names.append("Необходимо:")
for iterator in range(1,y+1):
    row_names.append('')

sheet2 = ipysheet.sheet(rows=y+1, columns=x+1,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(x+1):
    for iterator2 in range(y+1):
        if(iterator2==0 and iterator1==0):
            pass
        else:
            ipysheet.cell(iterator2,iterator1,0)
sheet2

Sheet(column_headers=['Запасы:'], columns=1, layout=Layout(height='auto', width='auto'), row_headers=['Необход…

In [20]:
df4 = ipysheet.to_dataframe(sheet2)
df4

,Запасы:
Необходимо:,None


## Найдите оптимальный план

In [21]:
col_names = []
row_names = []
col_names.append("Запасы:")
for iterator in range(1,x+1):
    col_names.append("Предприятие "+str(iterator))
row_names.append("Необходимо:")
for iterator in range(1,y+1):
    row_names.append('')

sheet3 = ipysheet.sheet(rows=y+1, columns=x+1,column_headers=col_names,row_headers=row_names,layout=Layout(height='auto', width='auto'),row_width='')
for iterator1 in range(x+1):
    for iterator2 in range(y+1):
        if(iterator2==0 and iterator1==0):
            pass
        else:
            ipysheet.cell(iterator2,iterator1,0)
sheet3

Sheet(column_headers=['Запасы:'], columns=1, layout=Layout(height='auto', width='auto'), row_headers=['Необход…

In [22]:
df5 = ipysheet.to_dataframe(sheet3)
df5

,Запасы:
Необходимо:,None


In [23]:
ans = widgets.Text(placeholder='Итоговая стоимость',description='',disabled=False)

display(ans)
def on_ans_clicked():
    x = int(a.value)
    if(x>0):
        a.value = ""
        print("Ответ верный")
    else:
        x=0
        print("Ответ не верный")
    
im = widgets.interact_manual(on_ans_clicked);

im.widget.children[0].description = 'Проверить'
im.widget.children[0].layout = Layout(width='30%')

Text(value='', placeholder='Итоговая стоимость')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [24]:
from ipywidgets import Layout
print("Введите ФИО, номер группы и код доступа")
name = widgets.Text(placeholder='ФИО',description='',disabled=False)
key = widgets.Text(placeholder='Код доступа',description='',disabled=False)
display(name,key)
def send_clicked():
    key_tmp = int(key.value)
    name_tmp = name.value
    if(key_tmp==1234):
        key.value = ""
        name.value = ""
        !jupyter nbconvert --config config.py 4_Transportation_Problem.ipynb
        print("Данные отправлены! Спасибо!")
    else:
        print("Неверный код доступа")
    
im = widgets.interact_manual(send_clicked);
im.widget.children[0].description = 'Отправить'
im.widget.children[0].layout = Layout(width='30%')

Введите ФИО, номер группы и код доступа


Text(value='', placeholder='ФИО')

Text(value='', placeholder='Код доступа')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…